## Mobile App for Lottery Addiction

In this project, we are going to contribute to the development of a mobile app by writing a couple of functions that are mostly focused on calculating probabilities. The app is aimed to both prevent and treat lottery addiction by helping people better estimate their chances of winning.

The app idea comes from a medical institute which is specialized in treating gambling addictions. The institute already has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities. For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that can answer users the following questions:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The original data set has data for 3,665 drawings, dating from 1982 to 2018, and can be found [here](https://www.kaggle.com/datascienceai/lottery-dataset).

The scenario we're following throughout this project is fictional — the main purpose is to practice applying probability and combinatorics (permutations and combinations) concepts in a setting that simulates a real-world scenario.

### Combinatoric functions

We'll start by writing two functions that we'll use often for this project:

- A function that calculates factorials; and
- A function that calculates combinations.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. The drawing is done without replacement, which means once a number is drawn, it's not put back in the set. 

To find the correct number of possible outcomes from a drawing without replacement we'll use the formula for combinations which takes only k objects from a group of n possible objects.

In [1]:
# Factorial func
def factorial(n):
    output = 1
    for i in range(n, 0, -1):
        output *= i
    return output

# Combinations func
def combinations(n, k):
    return factorial(n) / (factorial(k) * factorial(n-k))

### Probability of winning the big prize

In the 6/49 lottery, a player wins the big prize if the six numbers on their tickets match all the six numbers drawn. For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [2]:
def one_ticket_probability(list_):
    nr_outcomes = combinations(49, 6)
    probability = 1 / nr_outcomes
    percentage_probability = probability * 100
    print('Your chances to win the big prize with the numbers {} are {:.7f}%.\nIn other words, you have a 1 in {:,} chances to win.'.format(list_, percentage_probability, int(nr_outcomes)))

# Example output
one_ticket_probability([1, 2, 3, 4, 5, 6])

Your chances to win the big prize with the numbers [1, 2, 3, 4, 5, 6] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


### Historical Data Check for Canada Lottery

For the first version of the app, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018.

In [3]:
# Reading the dataset
import pandas as pd

can_loto = pd.read_csv('649.csv')
rows, columns = can_loto.shape
print('Nr of rows: {}. Nr of columns: {}'.format(rows, columns))
print('Sample output of the first rows:\n', can_loto.head(3), sep='\n')
print('Sample output of the last rows:\n', can_loto.tail(3), sep='\n')

Nr of rows: 3665. Nr of columns: 11
Sample output of the first rows:

   PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
0      649            1                0  6/12/1982               3   
1      649            2                0  6/19/1982               8   
2      649            3                0  6/26/1982               1   

   NUMBER DRAWN 2  NUMBER DRAWN 3  NUMBER DRAWN 4  NUMBER DRAWN 5  \
0              11              12              14              41   
1              33              36              37              39   
2               6              23              24              27   

   NUMBER DRAWN 6  BONUS NUMBER  
0              43            13  
1              41             9  
2              39            34  
Sample output of the last rows:

      PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
3662      649         3589                0  6/13/2018               6   
3663      649         3590                0  6/16/2

In [4]:
# func to extract winning numbers from dataset
def extract_numbers(row_):
    nr_columns = row_[4:10]
    return set(nr_columns)

winning_numbers = can_loto.apply(extract_numbers, axis=1)
print(winning_numbers.head(3))

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
dtype: object


In [5]:
# func to check historical occurence of a number
def check_historical_occurence(list_, series_):
    user_nr = set(list_)
    mask = series_ == user_nr
    nr_occurences = mask.sum()
    print('The number of times combination {} has occured in the past is {}.\nYour chances to win the big prize in the next drawing using the combination {} are 0.0000072%.\nIn other words, you have a 1 in 13,983,816 chances to win.'.format(list_, nr_occurences, list_))

# Example output
check_historical_occurence([1, 13, 45, 32, 30, 6], winning_numbers)
check_historical_occurence([33, 36, 37, 39, 8, 41], winning_numbers)

The number of times combination [1, 13, 45, 32, 30, 6] has occured in the past is 0.
Your chances to win the big prize in the next drawing using the combination [1, 13, 45, 32, 30, 6] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.
The number of times combination [33, 36, 37, 39, 8, 41] has occured in the past is 1.
Your chances to win the big prize in the next drawing using the combination [33, 36, 37, 39, 8, 41] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


### Probability of winning with multi-ticket

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning. We're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

In [6]:
def multi_ticket_probability(n):
    nr_outcomes = combinations(49, 6)
    probability = n / nr_outcomes
    percentage_probability = probability * 100
    chances = round(nr_outcomes / n)
    print('Your chances to win the big prize with {:,} different tickets are {:.6f}%.\nIn other words, you have a 1 in {:,} chances to win.'.format(n, percentage_probability, chances))
    
# Example output
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for test_input in test_inputs:
    multi_ticket_probability(test_input)
    print('------------------------')

Your chances to win the big prize with 1 different tickets are 0.000007%.
In other words, you have a 1 in 13,983,816 chances to win.
------------------------
Your chances to win the big prize with 10 different tickets are 0.000072%.
In other words, you have a 1 in 1,398,382 chances to win.
------------------------
Your chances to win the big prize with 100 different tickets are 0.000715%.
In other words, you have a 1 in 139,838 chances to win.
------------------------
Your chances to win the big prize with 10,000 different tickets are 0.071511%.
In other words, you have a 1 in 1,398 chances to win.
------------------------
Your chances to win the big prize with 1,000,000 different tickets are 7.151124%.
In other words, you have a 1 in 14 chances to win.
------------------------
Your chances to win the big prize with 6,991,908 different tickets are 50.000000%.
In other words, you have a 1 in 2 chances to win.
------------------------
Your chances to win the big prize with 13,983,816 dif

### Probability of winning small prizes: less winning numbers

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

We're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

In [17]:
def probability_less_6(n):
    nr_combinations = combinations(6, n)
    possible_loto_outcomes = combinations(43, (6-n)) # 43 possible winning numbers for each digit (44 - 1 exact number)
    succesful_outcomes = nr_combinations * possible_loto_outcomes
    probability = succesful_outcomes / combinations(49, 6)
    probability_percentage = probability * 100
    chances = round(combinations(49, 6) / succesful_outcomes)
    print('Your chances of having {} winning numbers with this ticket are {:.6f}%.\nIn other words, you have a 1 in {:,} chances to win.'.format(n, probability_percentage, chances))
    
# Example output
test_inputs = [2, 3, 4, 5]

for test_input in test_inputs:
    probability_less_6(test_input)
    print('------------------------')

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.
------------------------
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 57 chances to win.
------------------------
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
------------------------
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,201 chances to win.
------------------------


### Future work

Another possible features for a second version of the app may include:

- Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery").
- Combining the `one_ticket_probability()` and `check_historical_occurrence()` to output information on probability and historical occurrence at the same time.
- Create a function similar to `probability_less_6()` which calculates the probability of having at least two, three, four or five winning numbers. In this case, the number of successful outcomes for having at least four winning numbers is the sum of these three numbers:
    - The number of successful outcomes for having four winning numbers exactly.
    - The number of successful outcomes for having five winning numbers exactly.
    - The number of successful outcomes for having six winning numbers exactly.